In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import time

# Section 1: Monotone submodular maximization

Here we compare algorithms for (constrained) monotone submodular maximization. We will focus on cardinality constraints $\lvert S \rvert \leq k$, i.e. we seek to solve $\max_{S : \lvert S \rvert \leq k} f(S)$. We evaluate the algorithms on a function $f$ which is concave-over-modular. We implement two such functions: the first, `f_con_easy`, is generated from a modular function with random weights; the second, `f_con`, is initialized similarly, but 5 random elements are chosen to have especially high weight. This may affect the behavior of the algorithms we implement later...

In [ ]:
def concave_over_modular(weights, S):
    weights_arr = np.array(weights)
    return np.sqrt(np.sum(weights_arr[list(S)]))

np.random.seed(0)
n = 10000
V_con = set(range(n))

# version of concave-over-modular with random weights
weights = np.random.rand(n)
f_con_easy = lambda S: concave_over_modular(weights, S)

# version of concave-over-modular with some planted good items
weights_planted = weights.copy()
num_planted_weights = 5
inx_planted_weights = np.random.choice(list(range(len(weights))), size=num_planted_weights, replace=False)
weights_planted[inx_planted_weights] = 2 + 2 * np.random.rand(num_planted_weights)
f_con = lambda S: concave_over_modular(weights_planted, S)

The algorithms we consider access $f$ only through its marginal gains $\Delta(S, i) = f(S\cup\{i\}) - f(S)$. For some functions $f$, the marginal gains can be computed much faster than $f$ itself. For now, we give a helper function to compute $\Delta(S,i)$ for generic $f$:

In [ ]:
def marg_from_f(f):
    def marg(S, i):
        return f(S.union([i])) - f(S)
    
    return marg

Create marginal gains functions for both `f_con_easy` and `f_con`

In [ ]:
marg_con = marg_from_f(f_con)
marg_con_easy = marg_from_f(f_con_easy)

We wish to find a near-optimal subset $S$ of size $S \leq k$, where $k$ is set as:

In [ ]:
k = 10

## Greedy

Implement the basic greedy algorithm below. To debug, you can check your implementation against `lazy_greedy` (which we implemented for you) below

# <span style="color:red">TODO: implement</span>

In [ ]:
def greedy(marg, V, k):
    # implement greedy:
    # marg(S, i) returns f(S \union i) - f(S)
    # V is the ground set
    # k is the constraint on |S|

Check the object value achieved and the runtime for `greedy`:

In [ ]:
start = time.time()
S = greedy(marg_con, V_con, k)
print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f_con(S)))

## Lazy greedy

For your convenience, lazy greedy is implemented here:

In [ ]:
import heapq

def lazy_greedy(marg, V, k):
    if k >= len(V):
        return V.copy()
    
    # initialize heap to have (-1 times) upper bound on gain of each item
    marg_gains = [marg(set(), i) for i in V]
    neg_marg_gains = [-x for x in marg_gains]
    neg_marg_gains_tups = list(zip(neg_marg_gains, V))
    
    heapq.heapify(neg_marg_gains_tups)
    
    S = set()
    
    for _ in range(k):
        _, i = heapq.heappop(neg_marg_gains_tups)
        marg_gain_i = marg(S, i)
        
        # keep updating the marginal gains and re-inserting
        # until the top item of the heap is actually the best
        while marg_gain_i < -neg_marg_gains_tups[0][0]:
            heapq.heappush(neg_marg_gains_tups, (-marg_gain_i, i))
            _, i = heapq.heappop(neg_marg_gains_tups)
            marg_gain_i = marg(S, i)
            
        S.add(i)
    
    return S

Check the object value achieved and the runtime for `lazy_greedy`:

In [ ]:
start = time.time()
S = lazy_greedy(marg_con, V_con, k)
print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f_con(S)))

## Stochastic greedy

Implement stochastic greedy here. Stochastic greedy takes an additional parameter $s$, the number of candidate elements to sample each round

# <span style="color:red">TODO: implement</span>

In [ ]:
def stochastic_greedy(marg, V, k, s):
    # implement stochastic greedy:
    # marg(S, i) returns f(S \union i) - f(S)
    # V is the ground set
    # k is the constraint on |S|
    # s is the number of elements to sample each round

Check the object value achieved and the runtime for `stochastic_greedy`:

In [ ]:
start = time.time()
s = 100
S = stochastic_greedy(marg_con, V_con, k, s)
print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f_con(S)))

### More extensive timing and value comparison

Once you have working implementations for `greedy`, `lazy_greedy`, and `stochastic_greedy`, run the following code to compare their runtimes and objective values. 

In [ ]:
import pandas as pd

def compare_greedy_algs(f, V, s=100, num_stochastic_trials=10, k_max=10):
    # s: number of items that stochastic greedy samples

    marg = marg_from_f(f)

    times = []
    vals = []
    labels = []
    ks_list = []

    ks = list(range(1, k_max+1))
    for k in ks:
        start = time.time()
        S = greedy(marg, V, k)
        standard_time = time.time() - start        
        
        times.append(standard_time)
        vals.append(f(S))
        labels.append('Standard')
        ks_list.append(k)

        start = time.time()
        S = lazy_greedy(marg, V, k)
        lazy_time = time.time() - start

        times.append(lazy_time)
        vals.append(f(S))
        labels.append('Lazy')
        ks_list.append(k)

        for _ in range(num_stochastic_trials):
            start = time.time()
            S = stochastic_greedy(marg, V, k, s)
            stochastic_time = time.time() - start

            times.append(stochastic_time)
            vals.append(f(S))
            labels.append('Stochastic')
            ks_list.append(k)

        print('Done with k = {}/{}'.format(k, k_max))
        
    df = pd.DataFrame({
        'time': times,
        'val': vals,
        'label': labels,
        'k': ks_list
    })
    
    return df

Test out the algorithms on `f_con`, the objective function with planted high-weight elements:

In [ ]:
df = compare_greedy_algs(f_con, V_con)

In [ ]:
import seaborn as sns
plt.figure(figsize=(8,5))
sns.lineplot(x='k', y='time', hue='label', data=df)
plt.title('Runtime of three greedy variants (s); $f_{con}$')
plt.show()

plt.figure(figsize=(8,5))
sns.lineplot(x='k', y='val', hue='label', data=df)
plt.title('Objective value of three greedy variants; $f_{con}$')
plt.show()

Now test on the easier objective function `f_con_easy` where all weights are sampled uniformly:

In [ ]:
df = compare_greedy_algs(f_con_easy, V_con)

In [ ]:
import seaborn as sns
plt.figure(figsize=(8,5))
sns.lineplot(x='k', y='time', hue='label', data=df)
plt.title('Runtime of three greedy variants (s); $f_{con\_easy}$')
plt.show()

plt.figure(figsize=(8,5))
sns.lineplot(x='k', y='val', hue='label', data=df)
plt.title('Objective value of three greedy variants; $f_{con\_easy}$')
plt.show()

### Questions to ponder:
- What is the worst-case time complexity of greedy?
- When will lazy greedy be much faster than greedy? Can it ever be slower?
- How does the solution quality of greedy compare to that of lazy greedy?
- How does the solution quality and speed of stochastic greedy change as a function of the number of subsamples $s$?
- On what kinds of problems does stochastic greedy attain much worse solution quality than the other two algorithms?

# Section 2: Non-monotone submodular maximization

Now we will compare algorithms for non-monotone submodular maximization. 

First, we focus on the unconstrained case. 
To motivate this problem, we will take a brief detour and discuss graph cuts. 



### Graph cuts
The cut function $f(S)$ of a graph measures how strong the connection is between the nodes in $S$ and all other nodes (in $S^C$).
Formally, the cut function $f(S)$ is given by $f(S) = \sum_{i\in S} \sum_{j\in S^C} a_{ij}$; here $a_{ij}$ is the edge weight between nodes $i$ and $j$, or the $ij$-th component of the adjacency matrix $A$ (which you may assume is symmetric). 

The classic MAX CUT problem can be cast as an unconstrained non-monotone submodular maximization problem $\max_S f(S)$, where $f(S)$ is the cut function of the graph. 

Below we implement the cut function for you. First, the following utility code will sample a random graph:

In [ ]:
def erdos_renyi(n, p):
    A = np.random.binomial(n=1, p=p, size=(n, n))
    B = np.triu(A) + np.triu(A).T - np.diag(np.diag(A))
    
    return B # the adjacency matrix

def sbm(n, p, q):
    # p is within cluster, q between cluster
    # each cluster of size n
    
    A1 = erdos_renyi(n, p)
    A2 = erdos_renyi(n, p)
    B = np.random.binomial(n=1, p=q, size=(n,n))
    
    A_top = np.hstack([A1, B])
    A_bottom = np.hstack([B.T, A2])
    A = np.vstack([A_top, A_bottom])
    
    return A

`cut(A, S)` returns the value of the cut function when the adjacency matrix of the graph is `A`:

In [ ]:
def cut(A, S):
    n = A.shape[0]
    S_C = list(set(range(n)) - S)
    return np.sum(A[np.ix_(list(S), S_C)])

We sample a random graph and build its associated cut function. We also define `marg_cut`, the marginal gains function for $f$.

In [ ]:
# sample the graph and shuffle its nodes
A_clean = sbm(100, 0.05, 0.3)
inx = np.random.permutation(A_clean.shape[0])
A = A_clean[np.ix_(inx,inx)]

# instantiate the cut function for the sampled graph
V_cut = set(range(A.shape[0]))
f_cut = lambda S: cut(A, S)
marg_cut = marg_from_f(f_cut)

## Random selection

Now, we move onto optimization algorithms. The simplest approximation algorithm for unconstrained non-monotone submodular maximization is to return a uniform random set. Implement this:

In [ ]:
def random_selection(V):
    V_arr = np.array(list(V))
        
    to_take = np.random.randint(2, size=len(V), dtype=np.bool)
    S_arr = V_arr[to_take]
    
    return set(S_arr)

Check the mean object value achieved and the runtime. Random sampling should be fast!

In [ ]:
num_trials = 50
start = time.time()
cut_vals_random_selection = [f_cut(random_selection(V_cut)) for _ in range(num_trials)]
print('Took {0:.5f} seconds, mean objective value {1:.5f}'.format(time.time() - start, np.mean(cut_vals_random_selection)))

## Deterministic double greedy

Below we have implemented the deterministic version of the double greedy algorithm:

In [ ]:
def deterministic_double_greedy(marg, V):
    A = set()
    B = V.copy()
    
    for i in V:
        alpha = marg(A, i)
        beta = -marg(B.difference([i]), i) # f(B.difference([i])) - f(B)
        
        if alpha >= beta:
            A.add(i)
        else:
            B.remove(i)
            
    return A # = B

Check the objective value achieved and the runtime:

In [ ]:
start = time.time()
S = deterministic_double_greedy(marg_cut, V_cut)
cut_val_deterministic = f_cut(S)
print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f_cut(S)))

## Randomized double greedy

Now, implement the randomized version of double greedy. Instead of deterministically updating $A$ when $\alpha \geq \beta$, we update $A$ with probability $\alpha / (\alpha + \beta)$.

Depending on your particular implementation, you may have to clip $\alpha / (\alpha + \beta)$ to lie within $[0,1]$. 
* When and why can this happen?
* What does it mean when $\alpha = -\beta$?

Relate your answers to properties of $f$, e.g. submodularity, monotonicity...

# <span style="color:red">TODO: implement</span>

In [ ]:
def randomized_double_greedy(marg, V):
    # your code here

Check the object value achieved and the runtime:

In [ ]:
num_trials = 50
start = time.time()
cut_vals_randomized_double_greedy = [f_cut(randomized_double_greedy(marg_cut, V_cut)) for _ in range(num_trials)]
print('Took {0:.5f} seconds, mean objective value {1:.5f}'.format(time.time() - start, np.mean(cut_vals_randomized_double_greedy)))

How do these three approachs compare empirically? Remember,
* random selection obtains a 1/4-approximation
* randomized double greedy obtains a 1/2-approximation (optimal)
* deterministic double greedy obtains a 1/3-approximation

Below we plot histograms of the objective values achieved by each algorithm. (make sure to run all the code from the above section to populate the arrays)

In [ ]:
plt.figure(figsize=(8,5))

plt.hist(cut_vals_random_selection)
plt.hist(cut_vals_randomized_double_greedy)
plt.stem([cut_val_deterministic], [10], 'r')

plt.legend(['Random selection','Randomized double greedy','Deterministic double greedy'])
plt.xlabel('Objective value')
plt.ylabel('Frequency')
plt.title('Objective values $f(S)$ achieved by algorithms for\nnon-monotone, unconstrained submodular maximization')
plt.show()

## Faster marginal gain oracles (time-permitting)
Remember, we do not need $f$ itself, but rather the marginal gains $\Delta(S,i)$. (this holds true also for many of the approaches to submodular _minimization_). For cut functions, it is possible to compute $\Delta(S,i)$ much faster than simply evaluating $f(S\cup\{i\})$ and $f(S)$. Below, give a faster implementation of $\Delta(S,i)$:

# <span style="color:red">TODO: implement (time-permitting)</span>

In [ ]:
def build_marg_cut_from_A(A):

    def marg_cut_fast(S, i):
        # your code here; given an adjacency matrix A, a set S, and an element i not in S,
        # compute the marginal gains of the cut function 

    return marg_cut_fast
    
marg_cut_fast = build_marg_cut_from_A(A)
    
# sanity check -- these two marginal gains oracles should always return the same value
print(marg_cut_fast(set([1,2,3]), 4))
print(marg_cut(set([1,2,3]), 4))

Below we include code so you can compare the speed of the two implementations of $\Delta(S,i)$ as the size of the underlying graph changes.

In [ ]:
def graph_of_n_nodes(n):
    # sample the graph and shuffle its nodes
    n_half = int(n / 2)
    A_clean = sbm(n_half, 0.05, 0.3)
    inx = np.random.permutation(A_clean.shape[0])
    A = A_clean[np.ix_(inx,inx)]
    
    return A


def test_oracles_different_sized_graphs(n_list, n_trials=10):
    import time
    
    times_slow = []
    times_fast = []
    
    for n in n_list:
        A = graph_of_n_nodes(n)
        
        V = list(range(n-1)) # note we exclude the final node
        S_list = [random_selection(V) for _ in range(n_trials)]
        i = n - 1 # does not appear in any S above
        
        f_cut = lambda S: cut(A, S)
        marg_cut = marg_from_f(f_cut)
        marg_cut_fast = build_marg_cut_from_A(A)

        start = time.time()
        all_vals_slow = [marg_cut(S, i) for S in S_list]
        time_slow = time.time() - start
        times_slow.append(time_slow / n_trials)
        
        start = time.time()
        all_vals_fast = [marg_cut_fast(S, i) for S in S_list]
        time_fast = time.time() - start
        times_fast.append(time_fast / n_trials)
        
    plt.figure(figsize=(8,5))
    plt.plot(n_list, times_slow)
    plt.plot(n_list, times_fast)
    plt.legend(['Naive $\Delta(S,i)$', 'Fast $\Delta(S,i)$'])
    plt.xlabel('Number of nodes in graph')
    plt.ylabel('Average evaluation time (s)')
    plt.show()

In [ ]:
n_list = list(range(10, 501, 30))
test_oracles_different_sized_graphs(n_list)

# Section 3: Modeling

Here we focus on an image summarization task. We ingest a dataset (in this case MNIST), compute a similarity matrix between all datapoints, and use that to test a number of different formulations of summarizing the dataset.

We preprocess the dataset `X` by computing HOG features. Feel free to explore other features from `skimage`, or using e.g. raw pixel features. (Note: if you do this, you may need to adjust the length scale of the kernel below)

In [ ]:
import time

start = time.time()
from sklearn.datasets import fetch_openml
X_raw, y_full = fetch_openml('mnist_784', version=1, return_X_y=True)
print('Loading MNIST took {} seconds'.format(time.time() - start))

# efficient feature map for images
def hog_features(X_raw):
    from skimage.feature import hog

    hog_feat = [hog(np.reshape(X_raw[i,:], (28, 28))) for i in range(X_raw.shape[0])]
    X = np.array(hog_feat)
    
    return X

start = time.time()
X_full = hog_features(X_raw)
print('Feature extraction took {} seconds'.format(time.time() - start))

We seek a set $S$ of up to $k$ datapoints that well represent the entire dataset. There are a number of ways to frame this as maximizing $f(S)$ for various $f$. Suppose we have access to some similarity score $s(i,j)$ between the $i$-th and $j$-th datapoints, for example, the corresponding entry of a kernel matrix. Some possibilities of $f$ (in terms of $s$) are:
* Facility location
    - $f(S) = \sum_{i\in V} \max_{j \in S} s(i,j)$ 
    - montone submodular; nonnegative
* Sum coverage: 
    - $f(S) = \sum_{i\in V} \sum_{j\in S} s(i,j)$
    - monotone submodular; nonnegative
* Facility location with diversity penalty: 
    - $f(S) = \sum_{i\in V} \max_{j\in S} s(i,j) - \frac{1}{|V|} \sum_{i\in S} \sum_{j\in S} s(i,j)$
    - non-monotone submodular; nonnegative
* Log-subdeterminants:
    - $f(S) = \log\det K_{S,S}$
    - non-monotone submodular; may be negative
    
Explore these $f$ and other formulations you think up. Play around with the dataset, e.g. test different features of the images, or try out different levels of class imbalance.

Keep in mind the following questions:
* what desiderata does $f$ encode?
* what worst-case guarantees are there w.r.t. optimizing $f$?
* regardless of worst-case guarantees, do simple algorithms work well for $f$?
* how easy is $f$ to compute?
* how robust is $f$ to class imbalance?

### Defining the objective function $f(S)$

In [ ]:
# Instead of explicitly forming the kernel matrix, we compute similarity values only as needed using m = RBF()

from sklearn.gaussian_process.kernels import RBF
m = RBF(length_scale=1e0)

Below we define the objective functions discussed above. Each function, e.g. `facloc`, takes in the data matrix `X` and the kernel function `m`, and returns a function oracle `f(S)` that can be optimized using the algorithms above.

In [ ]:
# Facility location
def facloc(X, m):
    
    # for efficiency, we subsample V in the outer sum;
    # we fix the subsampling ahead of time so it is consistent across oracle calls
    n_sample = 1000
    n_sample = min(X.shape[0], n_sample)
    inx = np.random.choice(list(range(X.shape[0])), size=n_sample, replace=False)
    
    def f(S):
        if len(S) == 0:
            return 0
        
#         # if we weren't subsampling:
#         K_S = m(X[list(S),:], X)
        K_S = m(X[list(S),:], X[inx,:])
        return np.sum(np.max(K_S, axis=0))
    
    return f

# Sum coverage
def sum_coverage(X, m):

    # for efficiency, we subsample V in the outer sum;
    # we fix the subsampling ahead of time so it is consistent across oracle calls
    n_sample = 1000
    n_sample = min(X.shape[0], n_sample)
    inx = np.random.choice(list(range(X.shape[0])), size=n_sample, replace=False)
    
    def f(S):
        if len(S) == 0:
            return 0
        
        K_S = m(X[list(S),:], X[inx,:])
        return np.sum(K_S)
    
    return f

# Facility location with diversity penalty
def diverse_facloc(X, m):
    f_facloc = facloc(X, m)

    def f_diverse(S):
        if len(S) == 0:
            return 0
        
        K_SS = m(X[list(S),:])
        return -np.sum(K_SS) / X.shape[0]
    
    return lambda S: f_facloc(S) + f_diverse(S)

# Log determinant maximization
def dpp(X, m):
    def f(S):
        K_SS = m(X[list(S),:])
        (sign, logdet) = np.linalg.slogdet(K_SS)
        return logdet
    
    return f

### Visualizing the summary

Soon, we will optimize $f(S)$ in order to summarize the dataset. First, we provide some simple code for visualizing a candidate summary $S$ of images returned by the algorithm.

We print the labels that correspond to the selected images, and show the chosen images. Consider the following questions when evaluating the quality of the summary $S$:
* What is the distribution of labels in the summary? Does it match the overall dataset? 
* Do the images look typical? Do they look like outliers? 

In [ ]:
def show_summary(X_raw, y, S):
    # X_raw contains the raw images (not their features)
    
    S_list = np.array(list(S))
    inx_sort = np.argsort(y[S_list])
    S_sort = S_list[inx_sort]

    labels = ', '.join([str(int(x)) for x in y[S_sort]])
    print('Labels: {}'.format(labels))
    
    show_images = True
    if show_images:
        height = int(np.ceil(np.sqrt(len(S))))
        width = int(np.ceil(len(S) / height))
        
        fig, axs = plt.subplots(height, width, sharex=True, sharey=True, figsize=(2*height, 2*width))
        for c, i in enumerate(S_sort):
            X_i = np.reshape(X_raw[i,:], (28, 28))
            
            
            plt.subplot(width, height, c+1)
            plt.imshow(X_i)
            plt.axis('off')
        
        plt.show()

### Finding a good summary
We encourage you to try different objective functions $f$ and different optimization algorithms, e.g.
* `lazy_greedy` (is it still fast enough?)
* `stochastic_greedy` with varying number of subsamples $s$

In [ ]:
# set up the problem (ground set V, constraint k, objective f, marginal gains function)

V = set(range(X_full.shape[0]))
k = 10

# X_full is the full dataset of features; m is the kernel function
f = facloc(X_full, m)
marg = marg_from_f(f)

In [ ]:
start = time.time()
S = stochastic_greedy(marg, V, k, 100)
# S = lazy_greedy(marg, V, k)
print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f(S)))
# S = repeated_greedy(f, V, k, 3)

In [ ]:
show_summary(X_raw, y_full, S)

### Effect of class imbalance
Different objective functions $f(S)$ yield summaries that are affected by class imbalance in different ways. Below we provide some code to help you subsample the dataset to have a desired level of class imbalance. (Note: this may also be useful to subsample, in case optimization is taking too long on the full dataset)

In [ ]:
# method to subsample with specified weights for each class
def stratified_sampling(X_full, X_raw, y_full, class_weights, n_subsample):
    import numpy as np
    class_weights = np.array(class_weights) / np.sum(np.array(class_weights))

    class_nums = np.round(class_weights * n_subsample).astype(np.int32)
    all_class_labels = sorted(set(y_full))

    X_sampled_list = []
    X_raw_sampled_list = []
    y_sampled_list = []
    for label, count in zip(all_class_labels, class_nums):
        matching_label = (y_full == label)
        inxs_this_label = np.random.choice(np.where(matching_label)[0], size=count, replace=False)
        X_this_label = X_full[inxs_this_label, :]
        X_raw_this_label = X_raw[inxs_this_label, :]
        y_this_label = y_full[inxs_this_label]

        X_sampled_list.append(X_this_label)
        X_raw_sampled_list.append(X_raw_this_label)
        y_sampled_list.append(y_this_label)

    X_sampled = np.vstack(X_sampled_list)
    X_raw_sampled = np.vstack(X_raw_sampled_list)
    y_sampled = np.hstack(y_sampled_list)

    inxs_permuted = np.random.permutation(len(y_sampled))

    return X_sampled[inxs_permuted, :], X_raw_sampled[inxs_permuted, :], y_sampled[inxs_permuted]


# method to visualize the specified weights for each class
def visualize_class_weights(class_weights_imbalanced):
    df = pd.DataFrame({
        'label': list(range(len(class_weights_imbalanced))),
        'class_weights': class_weights_imbalanced
    })
    plt.figure(figsize=(8, 5))
    sns.barplot(x='label', y='class_weights', data=df)
    plt.xlabel('Class label')
    plt.ylabel('Relative frequency')
    plt.show()

In [ ]:
# Subsample so that each class is equally represented

n_subsample = 1000

class_weights_even = [0.1] * 10
X_balanced, X_raw_balanced, y_balanced = stratified_sampling(X_full, X_raw, y_full, class_weights_even, n_subsample)

# visualize the class weights
visualize_class_weights(class_weights_even)

In [ ]:
# Subsample so that classes are imbalanced

n_subsample = 1000

class_weights_imbalanced = [0.2, 0.1, 0.05, 0.05, 0.01, 0.01, 0.02, 0.01, 0.35, 0.2]
X_imbalanced, X_raw_imbalanced, y_imbalanced = stratified_sampling(X_full, X_raw, y_full, class_weights_imbalanced, n_subsample)

# visualize the class weights
visualize_class_weights(class_weights_imbalanced)

Set up the ground set and constraints:

In [ ]:
V = set(range(X_imbalanced.shape[0]))
k = 10

Try lots of objective functions on the imbalanced dataset:

In [ ]:
for objective in [facloc, diverse_facloc, sum_coverage, dpp]:
    print('Results for objective: {}'.format(objective.__name__))
    
    f = objective(X_imbalanced, m)
    marg = marg_from_f(f)

    start = time.time()
    # S = stochastic_greedy(marg, V, k, 100)
    S = lazy_greedy(marg, V, k)
    print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f(S)))
    show_summary(X_raw_imbalanced, y_imbalanced, S)

## (Optional) Repeated greedy
Several of the objective functions above are _not_ monotone (in particular, `diverse_facloc` and `dpp`). We wish to optimize these non-monotone submodular functions subject to a cardinality constraint, i.e. $\max_{S : |S|\leq k} f(S)$. So far, though, we have only discussed unconstrained non-monotone submodular maximization.

There are a handful of algorithms in the literature with guarantees for this problem setting. One of them is called "repeated greedy," which we have implemented for you below. The basic idea is to leverage both the strengths of standard greedy (for dealing with the constraint) with the strengths of double greedy (for dealing with non-monotonicity). The algorithm proceeds for several rounds; each round, 
* we first use greedy to select $k$ good items;
* then use double greedy to select the best subset of those (possibly all of them);
* finally, remove the chosen items from consideration and repeat.

At the end, we return the best subset chosen over all the rounds.

In [ ]:
def repeated_greedy(f, V, k, s=5):
    N = V.copy()
    
    S_list = []
    S_prime_list = []
    
    marg = marg_from_f(f)
    
    for i in range(s):
        S = lazy_greedy(marg, N, k)
        S_prime = deterministic_double_greedy(marg, S)
        
        S_list.append(S)
        S_prime_list.append(S_prime)
        
        N.difference_update(S)
        
    S_vals = [(f(S), S) for S in S_list + S_prime_list]
    S_vals_sorted = sorted(S_vals, reverse=True)
    best_tup = S_vals_sorted[0]
    
    return best_tup[1]

start = time.time()
repeated_greedy(f_con, V_con, 10, 5)
print('Took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f_con(S)))

Now compare the results achieved by `repeated_greedy` versus e.g. `lazy_greedy` for each of these non-monotone objectives. 
* Which algorithm achieves larger objective value? (is there any difference? why or why not?)
* Which algorithm yields a better summary?

In [ ]:
for objective in [diverse_facloc, dpp]:
    print('Results for objective: {}'.format(objective.__name__))
    
    f = objective(X_imbalanced, m)
    
    start = time.time()
    S = repeated_greedy(f, V, k)
    print('Repeated greedy took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f(S)))
    
    show_summary(X_raw_imbalanced, y_imbalanced, S)

    marg = marg_from_f(f)
    start = time.time()
    S = lazy_greedy(marg, V, k)
    print('In contrast, lazy greedy took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f(S)))
    print('\n\n')

In [ ]:
for objective in [diverse_facloc, dpp]:
    print('Results for objective: {}'.format(objective.__name__))
    
    f = objective(X_balanced, m)
    
    start = time.time()
    S = repeated_greedy(f, V, k)
    print('Repeated greedy took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f(S)))
    
    show_summary(X_raw_balanced, y_balanced, S)

    marg = marg_from_f(f)
    start = time.time()
    S = lazy_greedy(marg, V, k)
    print('In contrast, lazy greedy took {0:.5f} seconds, objective value {1:.5f}'.format(time.time() - start, f(S)))
    print('\n\n')